In [2]:
import pandas as pd
import plotly.graph_objects as go

In [4]:
df = pd.read_csv('/workspaces/Futures-First/BackTest/data/LE Feb25-Apr25 Calendar_Daily.csv')

df['Date'] = pd.to_datetime(df['Date'])

# Create a Serial Number column
df['Serial'] = range(len(df))

# Calculate Moving Averages
df['MA9'] = df['Close'].rolling(window=9).mean()
df['MA12'] = df['Close'].rolling(window=12).mean()

# Find intersections where MA9 crosses MA12
intersection_points = []
for i in range(1, len(df)):
    # Check for crossing points (sign change in the difference)
    if (df['MA9'][i] > df['MA12'][i] and df['MA9'][i-1] <= df['MA12'][i-1]) or \
       (df['MA9'][i] < df['MA12'][i] and df['MA9'][i-1] >= df['MA12'][i-1]):
        intersection_points.append((df['Serial'][i], df['Close'][i]))

# Create Candlestick Chart
fig = go.Figure(data=[go.Candlestick(x=df['Serial'],
                                     open=df['Open'],
                                     high=df['High'],
                                     low=df['Low'],
                                     close=df['Close'],
                                     name='Candlestick')])

# Add MA9 and MA12 to the chart
fig.add_trace(go.Scatter(x=df['Serial'], y=df['MA9'], mode='lines', name='MA9', line=dict(color='blue')))
fig.add_trace(go.Scatter(x=df['Serial'], y=df['MA12'], mode='lines', name='MA12', line=dict(color='red')))



# Update layout
fig.update_layout(title='Candlestick with MA9 and MA12 Intersections',
                  xaxis_title='Serial Number',
                  yaxis_title='Price',
                  xaxis_rangeslider_visible=False)

# Show interactive plot
fig.show()

In [9]:
import pandas as pd
import plotly.graph_objects as go

# Load data
df = pd.read_csv('/workspaces/Futures-First/BackTest/data/LE Feb25-Apr25 Calendar_Daily.csv')
df['Date'] = pd.to_datetime(df['Date'])

# Create a Serial Number column
df['Serial'] = range(len(df))

# Calculate Moving Averages
df['MA9'] = df['Close'].rolling(window=9).mean()
df['MA12'] = df['Close'].rolling(window=15).mean()

# Initialize variables
positions = []  # To keep track of open positions
trades = []     # To record completed trades
position = None # Current position (None, 'long', 'short')
entry_price = None
tick_size = 0.001

# Iterate over DataFrame to backtest strategy
for i in range(1, len(df)):
    # Check for crossing points (sign change in the difference)
    if df['MA9'][i] > df['MA12'][i] and df['MA9'][i-1] <= df['MA12'][i-1]:  # MA9 crosses above MA12
        if position == 'short':  # Close short position
            profit = (entry_price - df['Close'][i]) / tick_size
            trades.append(profit)
            position = None
        position = 'long'
        entry_price = df['Close'][i]
    elif df['MA9'][i] < df['MA12'][i] and df['MA9'][i-1] >= df['MA12'][i-1]:  # MA9 crosses below MA12
        if position == 'long':  # Close long position
            profit = (df['Close'][i] - entry_price) / tick_size
            trades.append(profit)
            position = None
        position = 'short'
        entry_price = df['Close'][i]

    # Check for trend-breaking moves
    if position == 'long' and df['Close'][i] < entry_price - tick_size:
        profit = (df['Close'][i] - entry_price) / tick_size
        trades.append(profit)
        position = None
    elif position == 'short' and df['Close'][i] > entry_price + tick_size:
        profit = (entry_price - df['Close'][i]) / tick_size
        trades.append(profit)
        position = None

# Calculate total profit
total_profit = sum(trades)

# Create Candlestick Chart
fig = go.Figure(data=[go.Candlestick(x=df['Serial'],
                                     open=df['Open'],
                                     high=df['High'],
                                     low=df['Low'],
                                     close=df['Close'],
                                     name='Candlestick')])

# Add MA9 and MA12 to the chart
fig.add_trace(go.Scatter(x=df['Serial'], y=df['MA9'], mode='lines', name='MA9', line=dict(color='blue')))
fig.add_trace(go.Scatter(x=df['Serial'], y=df['MA12'], mode='lines', name='MA12', line=dict(color='red')))

# Update layout
fig.update_layout(title='Candlestick with MA9 and MA12 Intersections',
                  xaxis_title='Serial Number',
                  yaxis_title='Price',
                  xaxis_rangeslider_visible=False)

# Show interactive plot
fig.show()

# Print total profit
print(f"Total Profit: {total_profit} ticks")


Total Profit: -1824.9999999999993 ticks


In [11]:
import pandas as pd
import plotly.graph_objects as go

# Load data
df = pd.read_csv('/workspaces/Futures-First/BackTest/data/LE Feb25-Apr25 Calendar_Daily.csv')
df['Date'] = pd.to_datetime(df['Date'])

# Create a Serial Number column
df['Serial'] = range(len(df))

# Calculate Moving Averages
df['MA9'] = df['Close'].rolling(window=9).mean()
df['MA12'] = df['Close'].rolling(window=12).mean()

# Initialize variables
positions = []     # To keep track of open positions
trades = []       # To record completed trades
entry_points = [] # List to store entry points for visualization
exit_points = []  # List to store exit points for visualization
position = None   # Current position (None, 'long', 'short')
entry_price = None
tick_size = 0.001

# Iterate over DataFrame to backtest strategy
for i in range(1, len(df)):
    # Check for crossing points (sign change in the difference)
    if df['MA9'][i] > df['MA12'][i] and df['MA9'][i-1] <= df['MA12'][i-1]:  # MA9 crosses above MA12
        if position == 'short':  # Close short position
            profit = (entry_price - df['Close'][i]) / tick_size
            trades.append(profit)
            exit_points.append((df['Serial'][i], df['Close'][i], 'short'))
            position = None
        # Open long position
        position = 'long'
        entry_price = df['Close'][i]
        entry_points.append((df['Serial'][i], df['Close'][i], 'long'))
    elif df['MA9'][i] < df['MA12'][i] and df['MA9'][i-1] >= df['MA12'][i-1]:  # MA9 crosses below MA12
        if position == 'long':  # Close long position
            profit = (df['Close'][i] - entry_price) / tick_size
            trades.append(profit)
            exit_points.append((df['Serial'][i], df['Close'][i], 'long'))
            position = None
        # Open short position
        position = 'short'
        entry_price = df['Close'][i]
        entry_points.append((df['Serial'][i], df['Close'][i], 'short'))

    # Check for trend-breaking moves
    if position == 'long' and df['Close'][i] < entry_price - tick_size:
        profit = (df['Close'][i] - entry_price) / tick_size
        trades.append(profit)
        exit_points.append((df['Serial'][i], df['Close'][i], 'long'))
        position = None
    elif position == 'short' and df['Close'][i] > entry_price + tick_size:
        profit = (entry_price - df['Close'][i]) / tick_size
        trades.append(profit)
        exit_points.append((df['Serial'][i], df['Close'][i], 'short'))
        position = None

# Calculate total profit
total_profit = sum(trades)

# Create Candlestick Chart
fig = go.Figure(data=[go.Candlestick(x=df['Serial'],
                                     open=df['Open'],
                                     high=df['High'],
                                     low=df['Low'],
                                     close=df['Close'],
                                     name='Candlestick')])

# Add MA9 and MA12 to the chart
fig.add_trace(go.Scatter(x=df['Serial'], y=df['MA9'], mode='lines', name='MA9', line=dict(color='blue')))
fig.add_trace(go.Scatter(x=df['Serial'], y=df['MA12'], mode='lines', name='MA12', line=dict(color='red')))

# Add entry and exit points to the chart
for point in entry_points:
    fig.add_trace(go.Scatter(x=[point[0]], y=[point[1]], mode='markers', name=f"Entry {point[2]}", 
                             marker=dict(symbol='triangle-up' if point[2] == 'long' else 'triangle-down', color='green' if point[2] == 'long' else 'red')))
for point in exit_points:
    fig.add_trace(go.Scatter(x=[point[0]], y=[point[1]], mode='markers', name=f"Exit {point[2]}", 
                             marker=dict(symbol='cross', color='blue')))

# Update layout
fig.update_layout(title='Candlestick with MA9 and MA12 Intersections',
                  xaxis_title='Serial Number',
                  yaxis_title='Price',
                  xaxis_rangeslider_visible=False)

# Show interactive plot
fig.show()

# Print total profit
print(f"Total Profit: {total_profit} ticks")


Total Profit: -299.9999999999992 ticks


In [18]:
import pandas as pd
import plotly.graph_objects as go

# Load data
df = pd.read_csv('/workspaces/Futures-First/BackTest/data/LE Feb25-Apr25 Calendar_Daily.csv')
df['Date'] = pd.to_datetime(df['Date'])

# Create a Serial Number column
df['Serial'] = range(len(df))

# Calculate Moving Averages
df['MA9'] = df['Close'].rolling(window=9).mean()
df['MA12'] = df['Close'].rolling(window=15).mean()

# Initialize variables
entry_points = []  # List to store entry points for visualization
exit_points = []   # List to store exit points for visualization
positions = []     # To keep track of open positions
entry_price = None
tick_size = 0.001
last_high = None  # Track last high for exit conditions
last_low = None   # Track last low for exit conditions

# Iterate over DataFrame to backtest strategy
for i in range(1, len(df)):
    # Check for crossing points (sign change in the difference)
    if df['MA9'][i] > df['MA12'][i] and df['MA9'][i-1] <= df['MA12'][i-1]:  # MA9 crosses above MA12
        if positions and positions[-1]['type'] == 'short':  # Close short position if open
            profit = (entry_price - df['Close'][i]) / tick_size
            exit_points.append((df['Serial'][i], df['Close'][i], 'short'))
            positions.pop()
        # Open long position
        positions.append({'type': 'long', 'entry_price': df['Close'][i]})
        entry_price = df['Close'][i]
        entry_points.append((df['Serial'][i], df['Close'][i], 'long'))
        last_high = df['High'][i]  # Update last high for exit condition
        last_low = df['Low'][i]    # Update last low for exit condition

    elif df['MA9'][i] < df['MA12'][i] and df['MA9'][i-1] >= df['MA12'][i-1]:  # MA9 crosses below MA12
        if positions and positions[-1]['type'] == 'long':  # Close long position if open
            profit = (df['Close'][i] - entry_price) / tick_size
            exit_points.append((df['Serial'][i], df['Close'][i], 'long'))
            positions.pop()
        # Open short position
        positions.append({'type': 'short', 'entry_price': df['Close'][i]})
        entry_price = df['Close'][i]
        entry_points.append((df['Serial'][i], df['Close'][i], 'short'))
        last_high = df['High'][i]  # Update last high for exit condition
        last_low = df['Low'][i]    # Update last low for exit condition

    # Check for trend-breaking moves
    if positions:
        if positions[-1]['type'] == 'long' and df['High'][i] < last_high:
            profit = (df['Close'][i] - entry_price) / tick_size
            exit_points.append((df['Serial'][i], df['Close'][i], 'long'))
            positions.pop()
            last_low = df['Low'][i]  # Update last low to current low
        elif positions[-1]['type'] == 'short' and df['High'][i] > last_high:
            profit = (entry_price - df['Close'][i]) / tick_size
            exit_points.append((df['Serial'][i], df['Close'][i], 'short'))
            positions.pop()
            last_high = df['High'][i]  # Update last high to current high

# Calculate total profit
total_profit = sum(profit for profit in trades if profit is not None)

# Create Candlestick Chart
fig = go.Figure(data=[go.Candlestick(x=df['Serial'],
                                     open=df['Open'],
                                     high=df['High'],
                                     low=df['Low'],
                                     close=df['Close'],
                                     name='Candlestick')])

# Add MA9 and MA12 to the chart
fig.add_trace(go.Scatter(x=df['Serial'], y=df['MA9'], mode='lines', name='MA9', line=dict(color='blue')))
fig.add_trace(go.Scatter(x=df['Serial'], y=df['MA12'], mode='lines', name='MA15', line=dict(color='red')))

# Add entry and exit points to the chart
for point in entry_points:
    fig.add_trace(go.Scatter(x=[point[0]], y=[point[1]], mode='markers', name=f"Entry {point[2]}", 
                             marker=dict(symbol='triangle-up' if point[2] == 'long' else 'triangle-down', color='green' if point[2] == 'long' else 'red')))
for point in exit_points:
    fig.add_trace(go.Scatter(x=[point[0]], y=[point[1]], mode='markers', name=f"Exit {point[2]}", 
                             marker=dict(symbol='cross', color='blue')))

# Update layout
fig.update_layout(title='Candlestick with MA9 and MA15 Intersections',
                  xaxis_title='Serial Number',
                  yaxis_title='Price',
                  xaxis_rangeslider_visible=False)

# Show interactive plot
fig.show()

# Print total profit
print(f"Total Profit: {total_profit} ticks")


Total Profit: -299.9999999999992 ticks
